In [109]:
import os
import datetime
import pprint
import xlrd
import dataset

In [31]:
def dated_file_name(extension, prefix):
    filename = prefix + "_" + datetime.datetime.now().isoformat() + "." + extension
    return filename

In [25]:
data_dir = "/Users/jjpr/dev/scratch/mkgu_scratch/data"

In [111]:
class MappingFile(object):
    def __repr__(self):
        return str((self.__class__.__name__, vars(self)))

In [38]:
mf_paths = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if x.endswith(".xls")]
mf_paths

['/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-4F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5H.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-8E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-5-SN0656.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-8-SN0657.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2466-1-SN0658.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2467-6_SN0649.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/5602-2 SN1025-000936.xls']

In [70]:
def is_comment_row(row):
    return str(row[0].value).lstrip().startswith("//")

In [132]:
cells = [mf.mapping_worksheet.row(i)[0] for i in [0, 20, -1]]
cells

[text:'// Auto generated mapfile', number:1.0, text:'']

In [133]:
[x.ctype for x in cells]

[1, 2, 1]

In [135]:
def row_col_valid(cell):
    if cell.ctype != 2: return False
    if type(cell.value) != float: return False
    if cell.value % 1 != 0: return False
    if cell.value < 0: return False
    if cell.value > 9: return False
    return True

[False, True, False]

In [136]:
def get_mapping(i, row):
    if not (row_col_valid(row[0]) and row_col_valid(row[1])):
        return None
    result = {
        "column": row[0].value, 
        "row": row[1].value, 
        "bank_name": row[2].value,
        "electrode_number_in_bank": row[3].value,
        "electrode_label": row[4].value,
        "index_in_mapping_sheet": i
    }
    return result

In [89]:
def get_array_id(row):
    stripped = row[0].value.strip()
    chopped = stripped[len("Cerebus mapping for array "):]
    squoze = ''.join(chopped.split())
    return squoze

'2467-6/SN0649'

In [138]:
def get_mapping_file(path):
    mf = MappingFile()
    mf.xls_path = path
    mf.workbook = xlrd.open_workbook(path)
    mf.mapping_worksheet = mf.workbook.sheet_by_name("Cerebus mapping")
    mf.comment_lines = set()
    mf.description_row = None
    mf.mappings = []
    for i, row in enumerate(mf.mapping_worksheet.get_rows()):
        if is_comment_row(row):
            mf.comment_lines.add(i)
        elif mf.description_row is None:
            mf.description_row = row
        else:
            mapping = get_mapping(i, row)
            if mapping is not None:
                mf.mappings.append(mapping)
    mf.array_id = get_array_id(mf.description_row)
    return mf

In [139]:
mfs = []

for path in mf_paths:
    mf = get_mapping_file(path)
    mfs.append(mf)

pprint.pprint([vars(mf) for mf in mfs])
# mfs

[{'array_id': '090609B-4F/SN0559',
  'comment_lines': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
  'description_row': [text:'Cerebus mapping for array 090609B-4F/SN0559',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      text:'elec',
                      empty:''],
  'mapping_worksheet': <xlrd.sheet.Sheet object at 0x10563d940>,
  'mappings': [{'bank_name': 'A',
                'column': 8.0,
                'electrode_label': 'elec99',
                'electrode_number_in_bank': '14',
                'index_in_mapping_sheet': 13,
                'row': 9.0},
               {'bank_name': 'A',
                'column': 7.0,
                'electrode_label': 'elec98',
                'electrode_number_in_bank': '10',
                'index_in_mapping_sheet': 14,
                'row': 9.0},
               {'bank_name': 'A',
                'column': 6.0,
    

In [118]:
dir(mfs[-1].mapping_worksheet.row(-1)[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_these',
 'ctype',
 'dump',
 'value',
 'xf_index']

In [121]:
type(mfs[-1].mapping_worksheet.row(-1)[0])

xlrd.sheet.Cell

In [129]:
type(mfs[-1].mapping_worksheet.row(-1)[0].value)

str

In [130]:
type(mfs[-1].mapping_worksheet.row(15)[0].value)

float

In [126]:
bool(mfs[-1].mapping_worksheet.row(-1)[0].value)

False

In [127]:
bool(mfs[-1].mapping_worksheet.row(15)[0].value)

True

In [128]:
bool(str(0))

True

In [140]:
[x.array_id for x in mfs]

['090609B-4F/SN0559',
 '090609B-5E/SN00547',
 '090609B-5F/SN0548',
 '090609B-5H/SN0549',
 '090609B-8E/SN0552',
 '2132-5/SN0656',
 '2132-8/SN0657',
 '2466-1/SN0658',
 '2467-6/SN0649',
 '5602-2/SN1025-000936']

In [106]:
try: 
    db
except NameError:
    db_file = os.path.join(data_dir, dated_file_name("db", "array_meta"))
    db_url = "sqlite:///" + db_file
    db = dataset.connect(db_url)

In [141]:
db

<Database(sqlite:////Users/jjpr/dev/scratch/mkgu_scratch/data/array_meta_2018-01-31T10:54:28.388709.db)>

In [142]:
tbl_array = db["array"]
tbl_mapping = db["mapping"]
for mf in mfs:
    array_dict = {"blackrock_id": mf.array_id, "mapping_xls":mf.xls_path}
    array_id = tbl_array.upsert(array_dict, keys=["blackrock_id"])
    for mapping in mf.mappings:
        insert = dict(mapping)
        insert["array_id"] = array_id
        tbl_mapping.insert(insert)